In [1]:
# Necessary imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.cross_validation import KFold
from sklearn.preprocessing import StandardScaler

%config InlineBackend.figure_format = 'svg'
%matplotlib inline

/Users/liuriguang/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data = []
for i in range(2,13):
    data.append(pd.read_csv(f"data/rental_data{i}.csv"))
df = pd.concat(data, ignore_index=True)
df.shape

(1262, 16)

In [3]:
df.columns

Index(['Unnamed: 0', 'Name', 'Type', 'Address', 'Built', 'Room Type', 'Bath',
       'Sqft', 'Price', 'Number of Schools', 'Average School Rating',
       'Median Rental Price', 'Median Listing Price', 'Community Features',
       'Unit Features', 'url'],
      dtype='object')

In [4]:
df["Room Type"].unique()

array(['3-bedroom', '1-bedroom', 'studio', '2-bedroom', '0-bedroom',
       '4-bedroom', '5-bedroom', '6-bedroom', '7-bedroom'], dtype=object)

In [5]:
# initialize a new column
df["Beds"] = -1

# Extract number of bedrooms from the Room Type column
temp = df["Room Type"]=="studio"
df.loc[temp,'Beds'] = 0
for i in range(8):
    temp = df["Room Type"]==f"{i}-bedroom"
    df.loc[temp,'Beds'] = i

df.sample(20)

,Unnamed: 0,Name,Type,Address,Built,Room Type,Bath,Sqft,Price,Number of Schools,Average School Rating,Median Rental Price,Median Listing Price,Community Features,Unit Features,url,Beds
936,173,NaN,Apartment,"404 E. Howell St. 404 E Howell St, Seattle, WA...",1920.0,1-bedroom,1.0,380.0,1350.0,3,3.33,2650.0,684990.0,NaN,NaN,https://www.realtor.com/realestateandhomes-det...,1
900,137,Echo Lake Apartments,Apartment,"1150 N 192nd St, Shoreline, WA 98133",2009.0,2-bedroom,1.0,896.0,1620.0,4,6.50,2395.0,642500.0,"['Balcony', 'Balcony', 'Business Center', 'Car...","['Air Conditioning', 'Disability Access', 'Dis...",https://www.realtor.com/realestateandhomes-det...,2
457,46,Elara,Apartment,"2134 Western Ave, Seattle, WA 98121",2018.0,2-bedroom,2.0,1064.0,4880.0,4,4.75,2650.0,684990.0,"['Air Conditioner', 'Balcony', 'Barbecue Area'...","['Air Conditioning', 'Cable or Satellite TV', ...",https://www.realtor.com/realestateandhomes-det...,2
872,109,East Howe Steps,Apartment,"1823 Eastlake Ave E, Seattle, WA 98102",2017.0,1-bedroom,1.0,474.0,1665.0,3,5.33,2650.0,684990.0,"['Balcony', 'Barbecue/Grill', 'Bright White Ca...","['Cable or Satellite TV', 'Ceiling Fan', 'Cour...",https://www.realtor.com/realestateandhomes-det...,1
1225,49,NaN,Condo/Townhome/Row Home/Co-Op,"1521 2nd Ave, Seattle, WA 98101",NaN,2-bedroom,2.0,1729.0,5995.0,3,3.33,2650.0,684990.0,NaN,Pets OK,https://www.realtor.com/realestateandhomes-det...,2
431,20,NaN,Other,"3850 50th Ave NE, Seattle, WA 98105",NaN,3-bedroom,2.0,2110.0,4750.0,5,7.00,2650.0,684990.0,NaN,NaN,https://www.realtor.com/realestateandhomes-det...,3
597,128,Urbana,Apartment,"1501 NW 56th St, Seattle, WA 98107",2014.0,1-bedroom,1.0,792.0,2202.0,5,7.40,2650.0,684990.0,"['Cats Allowed', 'Ceiling Fans', 'Clubhouse', ...","['Ceiling Fan', 'Dryer', 'View']",https://www.realtor.com/realestateandhomes-det...,1
157,157,West Ridge Park Apartments,Apartment,"7901 Delridge Way SW, Seattle, WA 98106",1990.0,2-bedroom,1.0,954.0,1665.0,5,3.80,2650.0,684990.0,"['BBQ Grill', 'Balcony', 'Barbecue Area', 'Bar...","['Cable or Satellite TV', 'Carport', 'Ceiling ...",https://www.realtor.com/realestateandhomes-det...,2
1193,17,Wellington Place,Apartment,"327 S 177th Pl, Burien, WA 98148",1992.0,2-bedroom,1.0,878.0,1450.0,5,4.80,1795.0,472500.0,"['24 Hour Availability', 'Balcony', 'Business ...","['Cable or Satellite TV', 'Ceiling Fan', 'Cour...",https://www.realtor.com/realestateandhomes-det...,2
1221,45,Patent 523,Apartment,"523 Broadway E, Seattle, WA 98102",2010.0,2-bedroom,2.0,1089.0,3169.0,3,3.33,2650.0,684990.0,"['Balcony', 'Carpet', 'Cats Allowed', 'Clubhou...","['Dishwasher', 'Disposal', 'Dryer', 'Microwave...",https://www.realtor.com/realestateandhomes-det...,2


In [6]:
# Deal with null values

df.dropna(subset=["Bath","Sqft","Price"], inplace=True)
df.dropna(subset=["Median Rental Price", "Median Listing Price"], inplace=True)
df["Built"].fillna(df["Built"].median(), inplace=True)

In [7]:
# Handle outliers
df = df[df["Price"] > 100]

In [8]:
df2 = df.drop_duplicates(subset=["Address"])

In [9]:
df2["Community Features"].unique().shape

(132,)

In [10]:
df2["Unit Features"].unique().shape

(140,)

In [11]:
community = df2["Community Features"].dropna()
unit = df2["Unit Features"].dropna()

In [12]:
unit = unit.replace("Pets OK","['Cats Allowed', 'Dogs Allowed']")
unit = unit.replace("Cats OK","['Cats Allowed']")

In [13]:
from collections import defaultdict
import ast

In [14]:
# Generate a function to select most popular features
def feature_selection(fea, occ):
    count = defaultdict(int)
    for f in fea:
        f = ast.literal_eval(f)
        for ff in f:
            count[ff] += 1

    popular = []        
    for k,v in count.items():
        if len(k) > 5 and v >= occ:
            popular.append(k)
    
    return popular

In [15]:
pop_comm = feature_selection(community, 5)

In [16]:
pop_unit = feature_selection(unit, 5)

In [17]:
df.fillna(value={"Community Features":"[]", \
                 "Unit Features":"[]"}, inplace=True) 

In [18]:
df.replace({'Unit Features': \
                {"Pets OK": "['Cats Allowed', 'Dogs Allowed']", \
                 "Cats OK": "['Cats Allowed']"}}, inplace=True)

In [19]:
df["community_count"] = 0
df["unit_count"] = 0
for index, row in df.iterrows():
    i = 0
    a = row["Community Features"]
    a = ast.literal_eval(a)
    for aa in a:
        if aa in pop_comm:
            i += 1
    df["community_count"][index] = i

/Users/liuriguang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [20]:
for index, row in df.iterrows():
    i = 0
    a = row["Unit Features"]
    a = ast.literal_eval(a)
    for aa in a:
        if aa in pop_comm:
            i += 1
    df["unit_count"][index] = i

/Users/liuriguang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [21]:
df.columns

Index(['Unnamed: 0', 'Name', 'Type', 'Address', 'Built', 'Room Type', 'Bath',
       'Sqft', 'Price', 'Number of Schools', 'Average School Rating',
       'Median Rental Price', 'Median Listing Price', 'Community Features',
       'Unit Features', 'url', 'Beds', 'community_count', 'unit_count'],
      dtype='object')

In [22]:
df1 = df[["Name", "Address", "Type", "Built", "Beds", "Bath", "Sqft",
         "Number of Schools", "Average School Rating",
         "Median Rental Price", "Median Listing Price",
         "community_count", "unit_count", "Price"]]
df1

,Name,Address,Type,Built,Beds,Bath,Sqft,Number of Schools,Average School Rating,Median Rental Price,Median Listing Price,community_count,unit_count,Price
1,NaN,"590 1st Ave S Apt 603, Seattle, WA 98104",Single Family Home,2014.0,1,1.0,636.0,5,5.40,2650.0,684990.0,0,2,2195.0
2,NaN,"2726 Franklin Ave E 2726 Franklin Ave E, Seatt...",Apartment,2014.0,1,1.0,740.0,4,7.50,2650.0,684990.0,1,0,1595.0
3,Helix Ellipse,"4751 12th Ave NE, Seattle, WA 98105",Apartment,2006.0,0,1.0,541.0,5,8.20,2650.0,684990.0,58,1,1658.0
4,Helix Ellipse,"4751 12th Ave NE, Seattle, WA 98105",Apartment,2006.0,1,1.0,694.0,5,8.20,2650.0,684990.0,58,1,1923.0
5,Helix Ellipse,"4751 12th Ave NE, Seattle, WA 98105",Apartment,2006.0,1,1.0,724.0,5,8.20,2650.0,684990.0,58,1,1883.0
6,Helix Ellipse,"4751 12th Ave NE, Seattle, WA 98105",Apartment,2006.0,1,1.0,731.0,5,8.20,2650.0,684990.0,58,1,1983.0
7,Helix Ellipse,"4751 12th Ave NE, Seattle, WA 98105",Apartment,2006.0,1,1.0,650.0,5,8.20,2650.0,684990.0,58,1,1818.0
8,Helix Ellipse,"4751 12th Ave NE, Seattle, WA 98105",Apartment,2006.0,1,1.0,764.0,5,8.20,2650.0,684990.0,58,1,1903.0
9,Helix Ellipse,"4751 12th Ave NE, Seattle, WA 98105",Apartment,2006.0,1,1.0,755.0,5,8.20,2650.0,684990.0,58,1,1858.0
10,Helix Ellipse,"4751 12th Ave NE, Seattle, WA 98105",Apartment,2006.0,1,1.0,680.0,5,8.20,2650.0,684990.0,58,1,1808.0


In [23]:
df1.drop(df1[df1.Sqft == 0].index, inplace=True)

/Users/liuriguang/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [53]:
df1

,Name,Address,Type,Built,Beds,Bath,Sqft,Number of Schools,Average School Rating,Median Rental Price,Median Listing Price,community_count,unit_count,Price
1,NaN,"590 1st Ave S Apt 603, Seattle, WA 98104",Single Family Home,2014.0,1,1.0,636.0,5,5.40,2650.0,684990.0,0,2,2195.0
2,NaN,"2726 Franklin Ave E 2726 Franklin Ave E, Seatt...",Apartment,2014.0,1,1.0,740.0,4,7.50,2650.0,684990.0,1,0,1595.0
3,Helix Ellipse,"4751 12th Ave NE, Seattle, WA 98105",Apartment,2006.0,0,1.0,541.0,5,8.20,2650.0,684990.0,58,1,1658.0
4,Helix Ellipse,"4751 12th Ave NE, Seattle, WA 98105",Apartment,2006.0,1,1.0,694.0,5,8.20,2650.0,684990.0,58,1,1923.0
5,Helix Ellipse,"4751 12th Ave NE, Seattle, WA 98105",Apartment,2006.0,1,1.0,724.0,5,8.20,2650.0,684990.0,58,1,1883.0
6,Helix Ellipse,"4751 12th Ave NE, Seattle, WA 98105",Apartment,2006.0,1,1.0,731.0,5,8.20,2650.0,684990.0,58,1,1983.0
7,Helix Ellipse,"4751 12th Ave NE, Seattle, WA 98105",Apartment,2006.0,1,1.0,650.0,5,8.20,2650.0,684990.0,58,1,1818.0
8,Helix Ellipse,"4751 12th Ave NE, Seattle, WA 98105",Apartment,2006.0,1,1.0,764.0,5,8.20,2650.0,684990.0,58,1,1903.0
9,Helix Ellipse,"4751 12th Ave NE, Seattle, WA 98105",Apartment,2006.0,1,1.0,755.0,5,8.20,2650.0,684990.0,58,1,1858.0
10,Helix Ellipse,"4751 12th Ave NE, Seattle, WA 98105",Apartment,2006.0,1,1.0,680.0,5,8.20,2650.0,684990.0,58,1,1808.0


In [60]:
df1_grouped = df1.groupby(["Address", "Sqft","Beds", "Bath",
                          "Type","Built","Number of Schools",
                          "Average School Rating","Median Rental Price",
                          "Median Listing Price","community_count",
                          "unit_count"])

In [62]:
df2 = df1_grouped.Price.mean().reset_index()

In [63]:
df2.shape

(1100, 13)

In [64]:
df2.to_csv("data/rental-data-clean2.csv")

In [65]:
df1.shape

(1156, 14)